# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys_mine2 import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/Output_Data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zhob,31.3411,69.4481,21.13,17,1,1.56,PK,1698331289
1,1,yellowknife,62.4560,-114.3525,-1.71,86,100,5.14,CA,1698331021
2,2,ilulissat,69.2167,-51.1000,0.01,64,75,7.20,GL,1698331293
3,3,iqaluit,63.7506,-68.5145,-4.15,79,100,6.17,CA,1698331295
4,4,puerto natales,-51.7236,-72.4875,5.25,65,40,9.77,CL,1698331298


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
#The "c" (color) parameter was sourced from hvplot.holoviz.org.
Map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    s= "Humidity",
    c= "City",
    frame_height=500
)

# Display the map
Map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"]<27)&(city_data_df["Max Temp"]>21)&(city_data_df["Wind Speed"]<4.5)&(city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_weather=ideal_weather.dropna(how="any")

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,bethel,41.3712,-73.4140,21.67,62,0,0.00,US,1698331310
14,14,haiku-pauwela,20.9219,-156.3051,23.36,77,0,2.57,US,1698331319
30,30,kapa'a,22.0752,-159.3190,22.99,80,0,3.60,US,1698331353
45,45,greenville,35.6127,-77.3663,23.40,83,0,0.00,US,1698331255
47,47,lihue,21.9789,-159.3672,22.58,80,0,3.60,US,1698331391
60,60,fort bragg,35.1390,-79.0060,24.90,79,0,0.45,US,1698331424
113,113,evans,33.5337,-82.1307,22.91,79,0,0.45,US,1698331544
114,114,wailua homesteads,22.0669,-159.3780,22.24,79,0,3.60,US,1698331546
142,142,port elizabeth,-33.9180,25.5701,24.16,64,0,3.09,ZA,1698331609
211,211,kill devil hills,36.0307,-75.6760,21.66,77,0,1.54,US,1698331628


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#The syntax of the Pandas copy function was sourced from a Stack Overflow Board.
hotel_df= ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,62,
14,haiku-pauwela,US,20.9219,-156.3051,77,
30,kapa'a,US,22.0752,-159.3190,80,
45,greenville,US,35.6127,-77.3663,83,
47,lihue,US,21.9789,-159.3672,80,
60,fort bragg,US,35.1390,-79.0060,79,
113,evans,US,33.5337,-82.1307,79,
114,wailua homesteads,US,22.0669,-159.3780,79,
142,port elizabeth,ZA,-33.9180,25.5701,64,
211,kill devil hills,US,36.0307,-75.6760,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat= hotel_df.loc[index, "Lat"]
    Lng= hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kapa'a - nearest hotel: Pono Kai Resort
greenville - nearest hotel: Quality Inn
lihue - nearest hotel: Kauai Palms
fort bragg - nearest hotel: Airborne Inn Lodging
evans - nearest hotel: Hyatt Place Augusta
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port elizabeth - nearest hotel: Waterford Hotel
kill devil hills - nearest hotel: Mariner Days Inn & Suites
kailua-kona - nearest hotel: Kona Seaside Hotel
san rafael - nearest hotel: Hotel Regional
westport - nearest hotel: Norwalk Inn and Conference Center
holualoa - nearest hotel: Kona Hotel
cabo san lucas - nearest hotel: Comfort Rooms
dawesville - nearest hotel: Baymont by Wyndham Thomasville
san juan - nearest hotel: Hotel San Francisco
talen - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,62,Hampton Inn Danbury
14,haiku-pauwela,US,20.9219,-156.3051,77,Inn At Mama's Fish House
30,kapa'a,US,22.0752,-159.3190,80,Pono Kai Resort
45,greenville,US,35.6127,-77.3663,83,Quality Inn
47,lihue,US,21.9789,-159.3672,80,Kauai Palms
60,fort bragg,US,35.1390,-79.0060,79,Airborne Inn Lodging
113,evans,US,33.5337,-82.1307,79,Hyatt Place Augusta
114,wailua homesteads,US,22.0669,-159.3780,79,Hilton Garden Inn Kauai Wailua Bay
142,port elizabeth,ZA,-33.9180,25.5701,64,Waterford Hotel
211,kill devil hills,US,36.0307,-75.6760,77,Mariner Days Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
#The hover_cols parameter was sourced from hvplot.holoviz.org.

HotelMap = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    s= "Humidity",
    c= "City",
    hover_cols=["Hotel Name","Country"],
    frame_height=420
)

# Display the map
HotelMap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)